In [1]:
import numpy as np
from matplotlib import pyplot as plt
from dftpy.ions import Ions
from ase.build import bulk
from ase.io import read
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal

from scipy.optimize import minimize

/home/vr371/.local/lib/python3.8/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')


#### Purpose 
Since we observed a pattern in $\rho_{0}$ we can learn it with the data we generated. To relate $\rho_{0}$ with the phases of Si and its size we built Ewal sum matrix descriptors for all the phases. 

#### Creating the database 
We create the Ewal sum matrix for the five phases of silicon.

In [2]:
from dscribe.descriptors import CoulombMatrix, SineMatrix, EwaldSumMatrix, SOAP
traj_Btin = []
tral_fcc = []
traj_bcc = []
traj_8cd = []
traj_7cd = []
Ewal_matrices = []
Sine_matrices = []
Sine_gradient = []
Soap_matrices = []

r = np.linspace(0.8, 1.4, 30)
for i, d in enumerate(r):
    si_Btin = Ions.from_ase(crystal('Si',[(0,0,0)],spacegroup=141,cellpar = [4.81,4.81,2.65, 90,90,90]))
    cell = si_Btin.get_cell()
    si_Btin.set_cell(cell * d, scale_atoms=True)
    
    si_fcc = Ions.from_ase(bulk('Si', 'fcc', a= 3.405, cubic=True))
    cell = si_fcc.get_cell()
    si_fcc.set_cell(cell * d, scale_atoms=True)
    
    si_bcc = Ions.from_ase(bulk('Si', 'bcc', a= 3.09, cubic=True))
    cell = si_bcc.get_cell()
    si_bcc.set_cell(cell * d, scale_atoms=True)
    
    si_8cd = Ions.from_ase(bulk('Si', 'diamond', a=5.43, cubic=True))
    cell = si_8cd.get_cell()
    si_8cd.set_cell(cell * d, scale_atoms=True)
  
    si = bulk('Si', 'diamond', a=5.43, cubic=True)
    si.pop(i=1)
    si_7cd = Ions.from_ase(si)
    cell = si_7cd.get_cell()
    si_7cd.set_cell(cell * d, scale_atoms=True)
    
    traj_Btin.append(si_Btin)
    tral_fcc.append(si_fcc)
    traj_bcc.append(si_bcc)
    traj_8cd.append(si_8cd)
    traj_7cd.append(si_7cd)
    phases = [si_Btin, si_fcc, si_bcc, si_8cd, si_7cd]

    sm = SineMatrix(n_atoms_max=8, flatten=False)
    sine_matrices = sm.create(phases, n_jobs=2)
    sine_gradient = np.gradient(sine_matrices)
    Sine_matrices.append(sine_matrices)
    Sine_gradient.append(sine_gradient)
    esm = EwaldSumMatrix(n_atoms_max=8)
    ewal_matrices = esm.create(phases, n_jobs=2)
    Ewal_matrices.append(ewal_matrices)

In [13]:
traj = traj_Btin + tral_fcc + traj_bcc + traj_8cd

In [14]:
soap = SOAP(
        species=["Si"],
        periodic=False,
        rcut=10.0,
        sigma=0.5,
        nmax=10,
        lmax=0,
    )
derivative, soap_matrix = soap.derivatives(
            traj
            )

In [7]:
np.asarray(Sine_gradient).shape

(30, 3, 5, 8, 8)

In [8]:
Fx_Btin = np.asarray(Sine_gradient)[:,0,0,:,:]

/tmp/ipykernel_270137/4155235967.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(derivative)[1].shape


(4, 4, 3, 55)

#### Save the data

In [3]:
np.asarray(Sine_matrices).shape
np.save('Sine_matrices.npy', np.asarray(Sine_matrices))
np.save('Sine_gradient.npy', np.asarray(Sine_gradient))

In [5]:
np.save('Ewal_matrices.npy', np.asarray(Ewal_matrices))
np.save('Soap_matrix.npy', soap_matrix, allow_pickle=True)

/home/valeria/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [24]:
np.save('traj_Btim.npy', traj_Btin)
np.save('traj_fcc.npy', tral_fcc)
np.save('traj_bcc.npy', traj_bcc)
np.save('traj_8cd.npy', traj_8cd)
np.save('traj_7cd.npy', traj_7cd)
np.save('traj.npy', traj)

/home/valeria/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
